In [3]:
import tensorflow as tf

# Check if GPU is available
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print("GPU available")
else:
    print("GPU not available")

GPU available


In [5]:
import csv

# Function to check if a row has consistent column count
def is_consistent_row(row, num_columns):
    return len(row) == num_columns

# Read the CSV file and remove rows with inconsistent column counts
input_file = 'cicddos2019_dataset.csv'
output_file = 'cicddos2019_dataset_clean.csv'

with open(input_file, 'r', newline='') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Get the header row and determine the number of columns
    header = next(reader)
    num_columns = len(header)
    writer.writerow(header)

    # Iterate over rows, filter out inconsistent rows, and write the consistent rows to the output file
    for row in reader:
        if is_consistent_row(row, num_columns):
            writer.writerow(row)


Data pre processing code.

In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load the cleaned dataset
df = pd.read_csv('cicddos2019_dataset_clean.csv')

# Preprocessing
# 1. Handling Missing Values (if any)
df.dropna(inplace=True)

encoder_protocol = LabelEncoder()
df['Protocol'] = encoder_protocol.fit_transform(df['Protocol'])

encoder_label = LabelEncoder()
df['Label'] = encoder_label.fit_transform(df['Label'])

# Update the 'Class' column values
df['Class'] = df['Class'].map({'Attack': 1, 'Benign': 0})


scaler = MinMaxScaler()
numerical_columns = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total',
                     'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                     'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean',
                     'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std',
                     'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
                     'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
                     'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
                     'Packet Length Std', 'Packet Length Variance', 'Avg Packet Size', 'Avg Fwd Segment Size',
                     'Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
                     'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
                     'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init Fwd Win Bytes',
                     'Init Bwd Win Bytes', 'Fwd Act Data Packets', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
                     'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Remove non-numeric columns
df = df.select_dtypes(include=['float64', 'int64'])

# Print the first 5 rows of the dataframe
print(df.head())


# Splitting the dataset into features (X) and target (y)
X = df.drop(columns=['Class'])  # Features
y = df['Class']  # Target variable

# Splitting the dataset into training (70%), testing (15%), and validation (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Printing the shapes of the resulting datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)
print("Validation set shape:", X_val.shape, y_val.shape)


   Unnamed: 0  Protocol  Flow Duration  Total Fwd Packets  \
0           0         2   1.805269e-03           0.000058   
1           1         2   8.333421e-09           0.000012   
2           2         2   3.916708e-07           0.000012   
3           3         2   8.943260e-04           0.000035   
4           4         2   8.939260e-04           0.000035   

   Total Backward Packets  Fwd Packets Length Total  Bwd Packets Length Total  \
0                     0.0                  0.000137                       0.0   
1                     0.0                  0.000053                       0.0   
2                     0.0                  0.000050                       0.0   
3                     0.0                  0.000092                       0.0   
4                     0.0                  0.000094                       0.0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  ...  \
0               0.012235               0.150634                0.1

Deep learning model by using even larger batch size and changing other few parameters to achieve accuracy with low learning time.

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import time

# Check GPU availability and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and memory growth is set.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, falling back to CPU.")

start_time = time.time()

# Define the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model with a lower learning rate
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with larger batch size and longer training time
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=2048)

# Calculate the time taken for training
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


GPU is available and memory growth is set.
Epoch 1/50
148/148 [==============================] - 6s 11ms/step - loss: 515221.4375 - accuracy: 0.6891 - val_loss: 149638.6406 - val_accuracy: 0.7726
Epoch 2/50
148/148 [==============================] - 1s 9ms/step - loss: 222152.6406 - accuracy: 0.6837 - val_loss: 88814.6016 - val_accuracy: 0.7730
Epoch 3/50
148/148 [==============================] - 1s 8ms/step - loss: 168094.0938 - accuracy: 0.6732 - val_loss: 59524.8984 - val_accuracy: 0.7730
Epoch 4/50
148/148 [==============================] - 1s 6ms/step - loss: 139814.5625 - accuracy: 0.6694 - val_loss: 48538.2617 - val_accuracy: 0.7731
Epoch 5/50
148/148 [==============================] - 1s 6ms/step - loss: 127157.8750 - accuracy: 0.6686 - val_loss: 36977.7617 - val_accuracy: 0.7730
Epoch 6/50
148/148 [==============================] - 1s 5ms/step - loss: 101314.6172 - accuracy: 0.6702 - val_loss: 32520.5020 - val_accuracy: 0.7726
Epoch 7/50
148/148 [=============================

Incorporating distributed strategy as a way to demo federated learning.

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import time

# Check GPU availability and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and memory growth is set.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, falling back to CPU.")

# Define the distributed strategy
strategy = tf.distribute.MirroredStrategy()

# Define the model inside the strategy scope
with strategy.scope():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model with a lower learning rate
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

start_time = time.time()

# Train the model with larger batch size and longer training time
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=2048)

# Calculate the time taken for training
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


GPU is available and memory growth is set.
Epoch 1/50
148/148 [==============================] - 3s 11ms/step - loss: 565311.3750 - accuracy: 0.6550 - val_loss: 150939.7812 - val_accuracy: 0.7726
Epoch 2/50
148/148 [==============================] - 1s 8ms/step - loss: 248582.8750 - accuracy: 0.6725 - val_loss: 83667.5938 - val_accuracy: 0.7730
Epoch 3/50
148/148 [==============================] - 1s 7ms/step - loss: 166762.7344 - accuracy: 0.6529 - val_loss: 61831.7500 - val_accuracy: 0.7730
Epoch 4/50
148/148 [==============================] - 1s 7ms/step - loss: 138403.5938 - accuracy: 0.6555 - val_loss: 50695.3633 - val_accuracy: 0.7731
Epoch 5/50
148/148 [==============================] - 1s 7ms/step - loss: 113733.1562 - accuracy: 0.6602 - val_loss: 38788.5234 - val_accuracy: 0.7731
Epoch 6/50
148/148 [==============================] - 1s 10ms/step - loss: 102309.1875 - accuracy: 0.6601 - val_loss: 30556.4082 - val_accuracy: 0.7731
Epoch 7/50
148/148 [============================

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import time

# Assuming the data is already preprocessed and split into X_train, X_val, X_test, y_train, y_val, y_test

# Define the distributed strategy
strategy = tf.distribute.MirroredStrategy()

# Define the model inside the strategy scope
with strategy.scope():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model with a lower learning rate
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

start_time = time.time()

# Train the model with larger batch size and longer training time
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=2048)

# Calculate the time taken for training
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


Epoch 1/20
148/148 [==============================] - 3s 10ms/step - loss: 747129.2500 - accuracy: 0.6050 - val_loss: 182662.8438 - val_accuracy: 0.7726
Epoch 2/20
148/148 [==============================] - 1s 8ms/step - loss: 260450.2812 - accuracy: 0.6567 - val_loss: 106290.6875 - val_accuracy: 0.7730
Epoch 3/20
148/148 [==============================] - 1s 7ms/step - loss: 163451.5000 - accuracy: 0.6468 - val_loss: 75903.5703 - val_accuracy: 0.7730
Epoch 4/20
148/148 [==============================] - 1s 7ms/step - loss: 144583.8438 - accuracy: 0.6483 - val_loss: 59863.9375 - val_accuracy: 0.7731
Epoch 5/20
148/148 [==============================] - 1s 8ms/step - loss: 112170.9688 - accuracy: 0.6499 - val_loss: 46508.1328 - val_accuracy: 0.7731
Epoch 6/20
148/148 [==============================] - 1s 7ms/step - loss: 106620.7578 - accuracy: 0.6511 - val_loss: 34198.5273 - val_accuracy: 0.7731
Epoch 7/20
148/148 [==============================] - 1s 7ms/step - loss: 91880.5156 - accu

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import time

# Assuming the data is already preprocessed and split into X_train, X_val, X_test, y_train, y_val, y_test

# Define the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model with a lower learning rate
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

start_time = time.time()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=2048)

# Calculate the time taken for training
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


Epoch 1/20
148/148 [==============================] - 3s 8ms/step - loss: 380838.2188 - accuracy: 0.6934 - val_loss: 80641.6094 - val_accuracy: 0.7730
Epoch 2/20
148/148 [==============================] - 1s 5ms/step - loss: 198630.6406 - accuracy: 0.6797 - val_loss: 49684.8867 - val_accuracy: 0.7730
Epoch 3/20
148/148 [==============================] - 1s 6ms/step - loss: 150976.6406 - accuracy: 0.6759 - val_loss: 37363.8320 - val_accuracy: 0.7730
Epoch 4/20
148/148 [==============================] - 1s 5ms/step - loss: 128453.2031 - accuracy: 0.6763 - val_loss: 29597.9863 - val_accuracy: 0.7731
Epoch 5/20
148/148 [==============================] - 1s 5ms/step - loss: 96213.4766 - accuracy: 0.6696 - val_loss: 23626.4453 - val_accuracy: 0.7731
Epoch 6/20
148/148 [==============================] - 1s 5ms/step - loss: 78474.6875 - accuracy: 0.6693 - val_loss: 18444.6270 - val_accuracy: 0.7730
Epoch 7/20
148/148 [==============================] - 1s 6ms/step - loss: 78159.6797 - accuracy: